In [1]:
%matplotlib inline

import numpy as np
import sys
from scipy import special, stats
from scipy.optimize import minimize, fminbound
import random
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import pickle

matplotlib.rcParams['figure.figsize'] = (20,10)
matplotlib.rcParams['font.size'] = 22
matplotlib.rcParams['legend.handlelength'] = 2
matplotlib.rcParams['lines.linewidth'] = 3

<div class="alert alert-danger"><b>Ce notebook est à remplir au fur et à mesure du cours et constituera votre rapport sur les trois premières journées de cours. Les questions entourées du symbole $\star$ sont à traiter en dehors du cours.</b></div>

<h1 class="text-center">EISC-211 : Méthodes de Monte-Carlo</h1>

<a id="SAW"></a><h1 style="border: 5px solid; text-align: center; padding: 10px">VII. Chemins auto-évitants</h1>

<div class="alert alert-success">On applique maintenant les algorithmes de simulation stochastique vus dans le module à un cas réaliste d'estimation complexe de somme, <i>i.e.</i> intégrale discrète.</div>

Soient $x = (x^1, x^2), y = (y^1, y^2) \in \mathbb{Z}^2$. $x$ et $y$ sont voisins, ce que l'on écrit $x \sim y$, si
$$\lvert x^1 - y^1 \rvert + \lvert x^2 - y^2 \rvert = 1.$$

On appelle <b>chemin de longueur</b> $n$ une suite $c = (c(0), \ldots, c(n))$ de $n{+}1$ voisins avec $c(0) = 0$. Un chemin $c$ est auto-évitant s'il ne visite aucun point deux fois ou plus. Soit $E_n$ l'ensemble des chemins de longueur $n$ et $A_n$ l'ensemble des chemins auto-évitants de longueur $n$. La Figure ci-dessous donne un exemple de chemin auto-évitant de longueur $n = 250$.

![title](chemin.png)

Le dénombrement des chemins auto-évitants (par exemple, combien y a-t-il de chemins auto-évitants de longueur $1000$ en dimension $3$?) est un problème théorique très important avec de nombreuses ramifications pratiques. D'autres caractéristiques des chemins auto-évitants jouent un rôle clef dans les applications, telles que le déplacement quadratique moyen:
$$
	D_n = \frac{1}{\lvert A_n \rvert} \sum_{c \in A_n} \lVert c(n) \rVert^2.
$$
Le déplacement quadratique moyen donne par exemple une indication du volume occupé par une protéine : l'objectif de ce BE est de trouver une estimation fiable de $D_{500}$. 

<h2> VII.1. Estimation par IIDMC </h2>

Si $C$ est une chemin auto-évitant tiré uniformément au hasard, alors l'équation précédente s'interprète de la manière suivante :
$$D_n = \mathbb{E}\left( \lVert C(n) \rVert^2 \right)$$
et donc la méthode de Monte--Carlo s'applique. Ainsi, un estimateur de $D_n$ est
$$\widehat D^{\text{MC}}_n = \frac{1}{N} \sum_{k=1}^N \lVert C_k(n) \rVert^2$$
où les $(C_k)$ sont i.i.d.\ uniformément réparties sur $A_n$.

<p class="bg-primary" style="padding:1em"><b>QUESTION VII.1.1.</b> Montrez que $\lvert E_n \rvert = 4^n$.</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.1.1.</b> </div>

On génère un chemin $C^E$ aléatoirement de la manière suivante:
<ol start="1">
  <li>$C^E(0) = 0$</li>
  <li> $k = 1, \ldots, n$, $C^E(k)$ est choisi uniformément au hasard parmi les $4$ voisins de $C^E(k-1)$</li>
</ol>
<p class="bg-primary" style="padding:1em"><b>QUESTION VII.1.2.</b> Utilisez la question précédente pour montrer que $C^E$ est uniformément réparti sur $E_n$, et déduisez-en une méthode de simulation par rejet pour générer un chemin auto-évitant uniformément au hasard.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.1.2.</b> </div>

Le code suivant permet de générer des marches aléatoires auto-évitantes par la méthode du rejet naïf.

In [ ]:
def estimation_SAW_IIDMC(longueur_chemin, taille_echantillon):
    """
    fonction qui génére taille_echantillon marche aléatoire auto-évitante
    avec du rejet naïf
    les chemins auto-évitants sont de longueur longueur_chemin en dimension 2
    la fonction renvoie la moyenne des extensions quadratiques, l'erreur relative
    et une information sur le nombre de rejets
    cette fonction écrit aussi le dernier chemin généré dans le fichier PATH.txt
    """
    N = taille_echantillon 
    n = longueur_chemin
    square_extension = np.zeros([N,1])
    nrejet = np.zeros([N,1])
    for step_i in range(N):
        i = 0
        cpt=1
        while((i+1) != n): # boucle while pour effectuer la methode de rejet afin de générer des marche aléatoires auto évitantes
            X = 0 # coordonnées courant de C^E
            Y = 0
            lattice = np.zeros([2*n + 1,2*n+1]) #tableau qui vaut 0 partout sauf sur les coordonnées de la chaine. Attention la coordonnée (0,0) est au centre du tableau.
            lattice[n, n] = 1 #la coordonnée (0,0) correspond à la coordonnée (n;n) de lattice. On l'initialise donc à 1.
            path = [[0, 0]] #Vecteur de sauvegarde de la chaine complète (X_0,X_1,...,X_n)
            for i in range(n): #on va générer sur une marche aléatoire sur E_n
                U=np.random.uniform(0,1,1)
                if U < 1/2:
                    X = X + 2 * (U < 1/4) - 1
                else:
                     Y = Y + 2 * (U < 3/4) - 1
        
                path_addition = [X,Y];
                path.append(path_addition)
                lattice_x = n  + X;
                lattice_y = n  + Y;
           
                if lattice[lattice_x, lattice_y] == 1:
                    i = 0
                    cpt=cpt+1
                    break #Si la chaine générée n'appartient pas à A_n, on arrête la construction de la chaine et on repart à zéro.
                    
                else:
                    lattice[lattice_x, lattice_y] = 1 #sinon on continue et on met à jour lattice
        
        square_extension[step_i,0] = (X**2 + Y**2) #Estimer le déplacement quadratique de la step_ième réalisation d'une SAW
        nrejet[step_i,0]=cpt
        
        
    with open('PATH.txt', 'wb') as f: ####petit code utile pour sauvegarder une chaine.... 
        pickle.dump(path, f)
    np.savetxt('LATTICE.txt', lattice, fmt='%i')
    
    
    estimation = np.mean(square_extension)
    erreur_relative = np.std(square_extension)/np.mean(square_extension)/np.sqrt(N)
    return[estimation, erreur_relative, N/np.sum(nrejet)]

def plot_SAW():
    """
    fonction qui trace le chemin actuellement dans le fichier PATH.txt et le trace
    """
    with open('PATH.txt', 'rb') as f:
        path = pickle.load(f)
    x=[]
    y=[]
    for j in range(len(path)):
        tmp=path[j]
        x.append(tmp[0])
        y.append(tmp[1])
    plt.plot(x, y,'-o')


<p class="bg-primary" style="padding:1em"><b>QUESTION VII.1.3.</b> Faites tourner les scripts ci-dessous pour visualiser certains chemins générés. Jusqu'à quelle longueur de chemin le temps d'exécution est-il raisonnable (moins de 10 secondes) ? Quelle est l'estimatoin de $D_5$ et l'erreur relative associée avec cette méthode ?</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.1.3.</b> </div>

In [ ]:
estimation_SAW_IIDMC(30, 1)
plot_SAW()

In [ ]:
n=5
N=10000
EST_IIDMC=estimation_SAW_IIDMC(n,N)
print("Déplacement quadratique estimé par IIDMC:",EST_IIDMC[0],"avec une erreur relative de ",EST_IIDMC[1])


<p class="bg-primary" style="padding:1em"><b>QUESTION VII.1.4.</b> Tracez l'évolution de $\widehat D_n^{\text{MC}}$ et de son erreur relative en fonction de $n$. Que se passe-t-il pour $n$ grand? Pour répondre à cette question, estimez par simulation la probabilité $\mathbb{P}(C^E \in A_n)$.</p>

In [ ]:
longeur_chemin_max=8
tab_long_chemin=range(2,longeur_chemin_max+1)
N=1000
EST_IIDMC = np.zeros([len(tab_long_chemin),3])
for i in range(len(tab_long_chemin)):
    n = tab_long_chemin[i]
    print(estimation_SAW_IIDMC(n,N))
    EST_IIDMC[i,:]=estimation_SAW_IIDMC(n,N)
    
fig = plt.figure()
plt.plot(tab_long_chemin, EST_IIDMC[:,0])
plt.xlabel("Longueur du chemin")
plt.ylabel("Déplacement quadratique")


fig = plt.figure()
plt.plot(tab_long_chemin, EST_IIDMC[:,2])
plt.xlabel("Longueur du chemin")
plt.ylabel("Probabilité de rejet")


<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.1.4.</b> </div>

<h2> VII.2. Estimation par échantillonnage préférentiel </h2>

Afin de diminuer la probabilité de rejet, on génère au hasard un chemin auto-évitant $C^{\text{IS}}$ de la manière suivante:
<ol start="1">
    <li>$C^{\text{IS}}(0) = 0$</li>
	<li>pour $k = 1, \ldots, n$  :</li>
	<ol start="a">
        <li>si $C^{\text{IS}}(k-1)$ a au moins un voisin libre, on en choisit un au hasard qui devient $C^{\text{IS}}(k)$</li>
        <li>sinon, on recommence à l'étape $1$</li>
     </ol>
</ol>

Pour un chemin auto-évitant $c$, on définit son poids $W(c)$ de la manière suivante :
$$ W(c) = d_0(c) \cdots d_{n-1}(c) \ \text{ avec } \ d_i(c) = \lvert \{ x \sim c(i): x \in (c(0), \cdots, c(i)) \} \rvert, $$
*i.e.* $d_i(c)$ est le nombre de voisins de $c(i)$ qui ne sont pas occupés par le chemin partiel $(c(0), \ldots, c(i))$.

<p class="bg-primary" style="padding:1em"><b>QUESTION VII.2.1.</b> Montrez que pour tout chemin auto-évitant $c \in A_n$, on a
$\mathbb{P} \left( C^{\text{IS}} = c \right) = \frac{1}{W(c)} $
et déduisez-en l'estimateur par échantillonnage préférentiel de $D_n$.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.2.1.</b> </div>


<p class="bg-primary" style="padding:1em"><b>QUESTION VII.2.2.</b> 
Quel est le problème de cet estimateur ? Justifiez le choix de l'estimateur suivant:
$$ \widehat D^{\text{IS}}_n = \frac{\displaystyle \sum_{k=1}^N W(C^{\text{IS}}_k) \lVert C^{\text{IS}}_k \rVert^2}{\displaystyle \sum_{k=1}^N W(C^{\text{IS}}_k)}$$
où les $(C^{\text{IS}}_k)$ sont i.i.d. et obtenus par l'échantillonneur préférentiel ci-dessus.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.2.2.</b> </div>


On admettra par la suite qu'un estimateur $\widehat S^{\text{IS}}_n$ de la variance de cet estimateur est donné par
$$ \widehat S^{\text{IS}}_n = \frac{ \displaystyle \sum_{k=1}^N \left( \lVert C^{\text{IS}}_k(n) \rVert^2 - \widehat D^{\text{IS}}_n \right)^2 W(C^{\text{IS}}_k)^2} {\displaystyle\left( \sum_{k=1}^N W(C^{\text{IS}}_k) \right)^2} $$

<p class="bg-primary" style="padding:1em"><b>QUESTION VII.2.3.</b> 
Complétez la fonction <code>estimation_SAW_IS</code> pour $n=100$ afin d'estimer $D_{100}$. Tracez l'évolution de $\widehat D_n^{\text{IS}}$ et de son erreur relative en fonction de $n$. Les résultats sont-ils cohérents avec ce que vous avez obtenu par rejet naïf ? Pour quelle valeur de $n$ l'estimation par échantillonnage préférentiel semble atteindre ses limites?
</p>

In [ ]:
def estimation_SAW_IS(longueur_chemin, taille_echantillon):
    N = taille_echantillon 
    n = longueur_chemin
    square_extension = np.zeros([N,1])
    weights = np.zeros([N,1]);
    moves = [[0,1], [0,-1], [-1,0], [1,0]]# déplacements possibles à partir de X_i=(X,Y)
    for step_i in range(N):
        i = 0
        while((i+1) != n):  #boucle while pour effectuer la methode de rejet afin de générer des marche aléatoires auto évitantes
            X = 0
            Y = 0 # initialisation de la marche
            weight = 1; # initialisation des poids. cela va correspondre au produit des d_i
            lattice = np.zeros([2*n + 1,2*n+1])
            lattice[n, n] = 1;   # tableau qui vaut 0 partout sauf sur les coordonnées de la chaine. Attention la coordonnée (0,0) est au centre du tableau.
            path = [[0, 0]]; #initialisation du %Vecteur Nx2 de sauvegarde de la chaine complète (X_0,X_1,...,X_n)
            for i in range(n):
                lattice_x = n  + X; lattice_y = n  + Y
                up = lattice[lattice_x, lattice_y+1]
                down = lattice[lattice_x, lattice_y-1]
                left = lattice[lattice_x-1, lattice_y]
                right = lattice[lattice_x+1, lattice_y]
                
                
                a = [1,1,1,1]
                b = [up,down,left,right]
                neighbors = [b_elt - a_elt for a_elt, b_elt in zip(a, b)]  # Calcul des voisins disponibles
                
                if np.sum(neighbors) == 0:  #aucun voisin n'est disponible
                    i = 0
                    break# Si la chaine générée n'appartient pas à A_n, on arrête la construction de la chaine et on repart à zéro.
            
                weight = weight * np.sum(neighbors);# mis à jour de weight à l'aide de d_i
                direction = np.min(np.where(np.random.uniform(0,1,1)<(np.cumsum(neighbors)/np.sum(neighbors))));
                coord=moves[direction]
                
                X = X + coord[0] #générer X_{i+1} connaissant X_i uniformément sur les voisins de X_i
            #non visités  et mettre à jour X, Y (utiliser le tableau moves de la ligne 5 par exemple)
                Y = Y + coord[1]
            
                path_addition = [X,Y]
                path.append(path_addition)
            
                lattice_x = n  + X
                lattice_y = n  + Y
                lattice[lattice_x,lattice_y] = 1
        
    
        weights[step_i,0] = weight
        square_extension[step_i,0] = (X**2 + Y**2) #Estimer l'extension de la step_ième réalisation d'une SAW
    
    
    estimation = np.mean(np.multiply(weights,square_extension)) / np.mean(weights)

    erreur_relative = np.sqrt(np.sum(np.multiply(np.power(square_extension-estimation,2),np.power(weights,2)))/np.power(np.sum(weights),2))/estimation

    
    with open('PATH.txt', 'wb') as f: ####petit code utile pour sauvegarder une chaine.... 
        pickle.dump(path, f)
    
    np.savetxt('LATTICE.txt', lattice, fmt='%i')
    
    return[estimation, erreur_relative]
    


In [ ]:
estimation_SAW_IS(100,1)   ###### petit code utile pour tracer une marche aléatoire auto-évitante
plot_SAW()

In [ ]:
longeur_chemin_max=110
tab_long_chemin=range(10,longeur_chemin_max,10)
N=1000
EST_IS = np.zeros([len(tab_long_chemin),2])
for i in range(len(tab_long_chemin)):
    n = tab_long_chemin[i]
    EST_IS[i,:]=estimation_SAW_IS(n,N)
    print(EST_IS[i,:], n)
fig = plt.figure()
plt.plot(tab_long_chemin, EST_IS[:,0])
plt.xlabel("Longeur du chemin")
plt.ylabel("Déplacement quadratique")


fig = plt.figure()
plt.plot(tab_long_chemin, EST_IS[:,1])
plt.xlabel("Longeur du chemin")
plt.ylabel("Erreur relative")
    


<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.2.3.</b> </div>

<h2> VII.3. Estimation par MCMC </h2>

On rappelle la description générale de l'algorithme de Metropolis-Hasting sur un espace d'état $\mathfrak{X}$ dénombrable, avec pour distribution cible $\pi$ et comme noyau de transition $K$. Etant donné $X_0$, on génére itérativement $X_n$ de la manière suivante :
<ol start="1">
	<li>Générer indépendamment $Y \sim K(X_n, \, \cdot)$ et $U$ uniformément réparti sur $[0,1]$;</li>
	<li>Choisir pour $X_{n+1}$:
	$$ X_{n+1} = Y \mathbf{1}(U \leq \varrho) + X_n \mathbf{1}(U > \varrho) \ \text{ avec } \ \varrho = \frac{\pi(Y)}{\pi(X_n)} \frac{K(Y, X_n)}{K(X_n, Y)}. $$ </li>
</ol>


<p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.1.</b> 
A quoi correspondent $X_n$, $\pi$ et $\mathfrak{X}$ dans le cas présent ? En déduire la valeur du rapport $\displaystyle \frac{\pi(Y)}{\pi(X_n)}$ ici.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.1.</b> </div>

L'algorithme du pivot est à ce jour l'algorithme le plus performant pour estimer $D_n$, son noyau $K$ est décrit de la manière suivante. Soit $c, c' \in E_n$ : alors $K(c, c')$ est la probabilité $\mathbb{P}(C = c')$ avec $C$ construit de la manière suivante:
<ol start="1">
	<li> Générer $U$ uniformément réparti sur $\{0, \ldots, n-1\}$; </li>
	<li> Tirer $I$ uniformément au hasard dans l'ensemble des isométries de $\mathbb{Z}^2$ qui laissent l'origine invariante;</li>
	<li> $C$ est obtenu de la manière suivante :
	$ (C(0), \ldots, C(U)) = (c(0), \ldots, c(U)) \ \text{ et } \ (C(U), \ldots, C(n)) = I_{c(u)}(c(U), \ldots, c(n)). $</li>
</ol>

Concernant le second point, on note que le groupe des isométries de $\mathbb{Z}^2$ est bien fini : il y en a $8$, à savoir l'identité, $3$ rotations et $5$ réflexions axiales. Dans la troisième étape, $I_x$ est la transformation obtenue en "centrant" $I$ en $x \in \mathbb{Z}^2$.

<p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.2.</b> 
Que vaut le rapport $\displaystyle \frac{K(c, c')}{K(c', c)}$ ici? Reformulez l'algorithme de Metropolis-Hastings dans ce cas.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.2.</b> </div>


 <p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.3.</b> 
Comment initialiser $C_0$?
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.3.</b> </div>

 <p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.4.</b> 
    Complétez la fonction <code>estimation_SAW_MCMC</code> pour $n=500$ afin d'estimer $D_{500}$.
</p>

In [ ]:
def rotate(chemin, centre, theta):
    che_arr=np.array(chemin)# choose a point which will be the center of rotation
    cc, s = np.cos(theta), np.sin(theta)
    R = np.matrix('{} {}; {} {}'.format(cc, -s, s, cc)) #define a rotation rotation matrix
    s=np.transpose(np.array(che_arr-centre))  #shift points in the plane so that the center of rotation is at the origin
    res=np.transpose(R.dot(s))+centre   #apply the rotation about the origin and shift again so the origin goes back to the desired center of rotation
    res=np.round(res,0) #pour revenir sur des valeurs entières
    m=res.tolist()
    return m

def reflexe(chemin, centre, type):
    che_arr=np.array(chemin)
    t1=np.array(chemin)
    c=centre 

    if(type==1):
        che_arr[:,0]=2*c[0]-che_arr[:,0]
        
    if(type==2):
        che_arr[:,1]=2*c[1]-che_arr[:,1]
        
    if(type==3):
        t1[:,0]=c[0]-(che_arr[:,1]-c[1])
        che_arr[:,1]=c[1]-(che_arr[:,0]-c[0])
        che_arr[:,0]=t1[:,0]
    if(type==4):
        t1[:,0]=c[0]+che_arr[:,1]-c[1]
        che_arr[:,1]=c[1]+che_arr[:,0]-c[0]
        che_arr[:,0]=t1[:,0]
    
    m=che_arr.tolist()

    return m


In [ ]:
def estimation_SAW_MCMC(longueur_chemin, taille_echantillon):
    N = taille_echantillon 
    n1 = longueur_chemin
    square_extension = np.zeros([N,1])

    with open('PATH.txt', 'rb') as f:   ##### initialisation du MCMC par une chaine issue de IIDMC ou IS.
        path = pickle.load(f)
        

    lattice = np.loadtxt('LATTICE.txt') ##### initialisation du MCMC par une chaine issue de IIDMC ou IS.
    n=len(path)-1;

    if(n1!=n):
        print("Problème de cohérence sur la longueur de la marche")
        
        
    lattice_x=[]
    lattice_y=[]
    for j in range(len(path)):
        tmp=path[j]
        lattice_x.append(n+tmp[0]) ####Calcul des coordonnées de lattice
        lattice_y.append(n+tmp[1])

    cpt2=0; ### compteur sur le nombre de SAW généré

    while(cpt2<N):
        k=random.randint(0,n-1) #choix du site pivot
        z=random.randint(1,8)# choix de la transformation isométrique: 2 réflexions diagonales, 2 réflexions axiales, rotations +- \pi/2 et pi, enfin l'identité.
        lattice = np.zeros([2*n + 1,2*n+1])
        if(z==1):
            resul=rotate(path[k:len(path)],path[k],np.pi/2)
            pathnew=path[0:k]+resul
            
    
        if(z==2):
            angle=np.pi
            resul=rotate(path[k:len(path)],path[k],angle)
            pathnew=path[0:k]+resul
    
        if(z==3):
            angle=3*np.pi/2
            resul=rotate(path[k:len(path)],path[k],angle)
            pathnew=path[0:k]+resul
    
        if(z==4):
            pathnew=path
   
        if(z==5):
            type=1;
            resul=reflexe(path[k:len(path)],path[k],type)
            pathnew=path[0:k]+resul
    
        if(z==6):
            type=2;
            resul=reflexe(path[k:len(path)],path[k],type)
            pathnew=path[0:k]+resul
    
        if(z==7):
            type=3;
            resul=reflexe(path[k:len(path)],path[k],type)
            pathnew=path[0:k]+resul
        
        if(z==8):
            type=4;
            resul=reflexe(path[k:len(path)],path[k],type)
            pathnew=path[0:k]+resul
            
            
        pathnew_arr=np.array(pathnew)
        lattice_x_new = n  + pathnew_arr[:,0]
        lattice_y_new = n  + pathnew_arr[:,1]
        
        for j in range(len(lattice_x_new)) :
            lattice[lattice_x_new[j],lattice_y_new[j]]=lattice[lattice_x_new[j],lattice_y_new[j]]+1
            
        if(np.amax(lattice)<2):     #Si la chaine générée n'appartient pas à A_n, on revient à la chaine précédente
            pathnew_arr=np.array(pathnew)
            pathnew_arr=pathnew_arr-pathnew_arr[0,:]                
            path=pathnew_arr.tolist()
          

        tmp=path[-1]                                                     
        square_extension[cpt2,0] = (tmp[0]**2 + tmp[1]**2) # calcul de l'extension quadratique
        cpt2=cpt2+1
        
        
    with open('PATH.txt', 'wb') as f: ####petit code utile pour sauvegarder une chaine.... 
        pickle.dump(path, f)
    
    np.savetxt('LATTICE.txt', lattice, fmt='%i')
    
    estimation= np.mean(square_extension)
    return estimation

In [ ]:
exten_IS= estimation_SAW_IS(100, 10000)
print("Déplacement quadratique moyen estimé par IS:",exten_IS[0],"avec une erreur relative de",exten_IS[1])
exten_MCMC= estimation_SAW_MCMC(100, 10000)
print("Déplacement quadratique moyen estimé par MCMC:",exten_MCMC)


In [ ]:
estimation_SAW_IS(500,1)
estimation_SAW_MCMC(500,100)   ###### petit code utile pour tracer une marche aléatoire auto-évitante
with open('PATH.txt', 'rb') as f:
    path = pickle.load(f)

lattice = np.loadtxt('LATTICE.txt', dtype=int)
x=[]
y=[]
for j in range(len(path)):
    tmp=path[j]
    x.append(tmp[0])
    y.append(tmp[1])
    
plt.plot(x, y,'-o')

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.4.</b> </div>

<p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.6.</b> 
Vérifiez la conjecture suivante $D_n\sim Dn^{\frac{3}{2}}$ quand $n\rightarrow +\infty$ où $D$ est une constante à déterminer.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.6.</b> </div>

 <p class="bg-primary" style="padding:1em"><b>QUESTION VII.3.5.</b> 
Comparez en fonction de la dimension les trois estimateurs de $D_n$ proposés dans ce BE.
</p>

<div class="alert alert-warning"><b>REPONSE A LA QUESTION VII.3.5.</b> </div>